<a href="https://colab.research.google.com/github/janzika/MATH3261-5285/blob/main/baroclinic_instability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://raw.githubusercontent.com/climate-in-the-cloud/workshop/master/agulhas-rings.png)
*Figure: Baroclinic instability is responsible for the formation of ocean eddies like the ones shown here off the coast of South Africa. Credit: [NASA Perpetial Ocean](https://youtu.be/CCmTY0PKGDs).*

# Lab 5: Baroclinic instability

In this lab we will examine baroclinic instability in a two-layer shallow water QG model. We will examine:

- counter-propagating Rossby waves
- coupling of the two layers via vortex stretching/squeezing
- the role of the baroclinic shear velocity and the condition for instability

By the end of this lab you will be able to:

- describe the evolution of small perturbations to the two-layer QG model qualitatively and in terms of energetics
- relate the base state parameters and initial conditions to the resulting instability
- predict for the behaviour of the model under new parameters / initial conditions and devise experiments to test these predictions

To use this notebook, you will need Python 3 and the latest version of [Dedalus](http://dedalus-project.org/) installed on your local machine.

If you do not have Python 3 and Dedalus, you can run the lab on [Google Colaboratory](http://colab.research.google.com).  

## Google Colaboratory

You can run this notebook in your browser using [Google Colaboratory](http://colab.research.google.com/) (or "Colab", for short). You will need a (free) [Google Account](https://myaccount.google.com/intro) to run the notebooks. If you would prefer not to sign up, you can still read the notebook, but you won't be able to run the code cells.

Remember that the keyboard shortcut to execute a code cell is **CMD/CTRL+ENTER**, while **SHIFT+ENTER** will move you to the next cell. You can also click the play button to the left of each cell.

Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI)  to learn more, or check out this [Overview of Colab Features](https://colab.research.google.com/notebooks/basic_features_overview.ipynb).

If you would like to mount your Google Drive in Colab, copy and paste the following code snippet:
```
from google.colab import drive
drive.mount('/content/drive')
!ls drive/'My Drive'
```

You can also save your notebook to Google Drive, or download a copy of the notebook file, using the **File Menu** in the top left corner.

# Background

This lab builds upon lecture materials from Topic 4: Quasigeostrophy. Before and after the lab you may like to familiarize yourself with the lecture [videos](https://moodle.telt.unsw.edu.au/mod/url/view.php?id=3611931), [notes](https://moodle.telt.unsw.edu.au/mod/folder/view.php?id=3603785), and [slides](https://moodle.telt.unsw.edu.au/mod/folder/view.php?id=3603788).

## Two-layer QG equations

**Video:** [Lecture 9.1 The two-layer model](https://vimeo.com/531496691)

We will model baroclinic instability using a two-layer quasigeostrophic model with a rigid lid and a flat bottom. In the lectures, we saw that this model is governed by the following equations of motion

$$
\frac{D q_1}{D t} = 0, \qquad \frac{D q_2}{D t} = 0,
$$

where $q_1$ and $q_2$ are the QG potential vorticity in the upper and lower layers, respectively,

$$
q_1 = \nabla^2 \psi_1 + \beta y + \frac{1}{2} k_d^2 \left( \psi_2 - \psi_1 \right),
$$

$$
q_2 = \nabla^2 \psi_2 + \beta y - \frac{1}{2} k_d^2 \left( \psi_2 - \psi_1 \right),
$$

Here, $k_d$ is the inverse of the Rossby deformation scale, $k_d = 1/L_d$.




## Base state and perturbations

**Video:** [Lecture 9.5 Baroclinic instability](https://vimeo.com/531512376)

We prescribe a base state with a velocity of $(U, 0)$ in the upper layer and $(-U,0)$ in the lower layer

$$
\overline{\psi_1} = - U y, \qquad \overline{\psi_2} = U y.
$$

This gives a base state potential vorticity of

$$
\overline{q_1} = \left( \beta + k_d^2 U \right) y, \qquad \overline{q_2} = \left( \beta - k_d^2 U \right) y.
$$

We can see that the potential vorticity gradient in the two layers will have opposite sign only if $k_d^2 U > \beta$.

Now consider small perturbations to the base state streamfunction and potential vorticity

$$
\psi_1 = \overline{\psi_1} + \psi_1', \qquad q_1 = \overline{q_1} + q_1', \qquad \text{where} \qquad q_1' = \nabla^2 \psi_1' + \frac{1}{2} k_d^2 \left( \psi_2' - \psi_1' \right),
$$

$$
\psi_2 = \overline{\psi_2} + \psi_2', \qquad q_2 = \overline{q_2} + q_2', \qquad \text{where} \qquad q_2' = \nabla^2 \psi_2' - \frac{1}{2} k_d^2 \left( \psi_2' - \psi_1' \right),  
$$

Substituting this into the equations of motion in each layer and neglecting nonlinear terms gives

$$
\left( \frac{\partial}{\partial t} + U \frac{\partial}{\partial x} \right) q_1' + \frac{\partial \psi_1'}{\partial x} \left( \beta + k_d^2 U \right) = 0,
$$

$$
\left( \frac{\partial}{\partial t} - U \frac{\partial}{\partial x} \right) q_2' + \frac{\partial \psi_2'}{\partial x} \left( \beta - k_d^2 U \right) = 0,
$$


## Further reading

- O'Brien, E. (2019) [Balancing the Potential Vorticity Seesaw: The bare essentials of baroclinic instability](https://link.springer.com/article/10.1007/s41748-019-00128-7). *Earth Systems and Environments* 3: 341-351.

- Keating, S. (2014) Baroclinic Instability. Lecture at the 2014 ARCCSS winter school [(YouTube link)](https://youtu.be/yhbJhB4-NDo)

# Experimental set-up

## Installing Dedalus

We start by installing Dedalus on the Colab virtual machine. This might take a few minutes.

In [ ]:
# Set environment variables for best performance
%env OMP_NUM_THREADS=1
%env NUMEXPR_MAX_THREADS=1

# Minimize logging output
import logging
logging.disable(logging.DEBUG)

# Check if running on google colab
import os
using_google_colab = bool(os.getenv("COLAB_RELEASE_TAG"))

# Check for Dedalus
try:
    import dedalus.public as de
    print("Dedalus already installed :)")
except:
    print("Dedalus not installed yet.")
    if using_google_colab:
        print("Installing for Google Colab.")
        print()
        # Step 1: Install FFTW
        !apt-get install libfftw3-dev
        !apt-get install libfftw3-mpi-dev
        # Step 2: Set paths for Dedalus installation
        import os
        os.environ['MPI_INCLUDE_PATH'] = "/usr/lib/x86_64-linux-gnu/openmpi/include"
        os.environ['MPI_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
        os.environ['FFTW_INCLUDE_PATH'] = "/usr/include"
        os.environ['FFTW_LIBRARY_PATH'] = "/usr/lib/x86_64-linux-gnu"
        # Step 3: Install Dedalus using pip
        !pip3 install cython "mpi4py<4.0" numpy setuptools wheel
        !CC=mpicc pip3 install --no-cache --no-build-isolation http://github.com/dedalusproject/dedalus/zipball/v2_master/
        !pip3 install -q ipympl
        # Step 4: Check installation
        print()
        try:
            import dedalus.public as de
            print("Dedalus successfully installed :)")
        except:
            print("Error installing Dedalus :(")
            raise
    else:
        print("See website for installation instructions:")
        print("https://dedalus-project.readthedocs.io/en/latest/pages/installation.html")

# Setup interactive matplotlib
if using_google_colab:
    from google.colab import output
    output.enable_custom_widget_manager()

## Libraries

We start by importing the usual libraries for Dedalus, Numpy, and Matplotlib.

In [ ]:
from dedalus import public as de
import numpy as np
import matplotlib.pyplot as plt
from dedalus.extras import flow_tools

## Basis and domains

We will work in a square domain. For simplicity, we will use periodic boundary conditions on each side of the domain. We do this by choosing the basis functions in the $x$ and $y$ directions to be Fourier (sines and cosines).

In [ ]:
# Domain size, resolution, and integration time
Lx, Ly = 2*np.pi, 2*np.pi
Nx, Ny = 64, 64
T = 10.01

# Basis functions
x_basis = de.Fourier('x', Nx, interval=(-Lx, Lx), dealias=3/2)
y_basis = de.Fourier('y', Ny, interval=(-Ly, Ly), dealias=3/2)
domain  = de.Domain([x_basis,y_basis], np.float64)

# Problem parameters

The problem parameters are
- `beta`: the planetary vorticity gradient $\beta$
- `U`: the background shear flow strength (eastward) $U$ in the upper layer and $-U$ in the lower layer
- `Kd`: the inverse deformation length $K_d = 1/L_d$
- `D`: diffusivity

To start, we will look at the case with no shear, $U = 0$, and no vortex stretching or squeezing, $K_d = 0$. The problem then reduces to two shallow water layers that do not interact.

In [ ]:
# problem parameters

beta = 1.    # planetary vorticity gradient
D    = 5e-4  # diffusivity

# Change these parameters to try different experiments

U    = 1.    # shear flow strength
Kd   = 2.    # deformation wavenumber

## Initial value problem

Now we need to tell Dedalus about the domain, variables, parameters, and equations. (We don't need to worry about boundary conditions because they are built into our choice of Fourier basis modes.)

We will define the problem in almost exactly the same way as we did when we simulated Rossby waves: the only difference here is that we now have *two* shallow water layers to simulate.

In [ ]:
# create the initial value problem
problem = de.IVP(domain, variables=['psi1','psi2'])

# declare the problem parameters
problem.parameters['beta'] = beta
problem.parameters['Kd']   = Kd
problem.parameters['U']    = U
problem.parameters['D']    = D

# declare substitutions
problem.substitutions['q1']     = "d(psi1,x=2) + d(psi1,y=2) + Kd**2*(psi2-psi1)/2"
problem.substitutions['q2']     = "d(psi2,x=2) + d(psi2,y=2) - Kd**2*(psi2-psi1)/2"
problem.substitutions['J(f,g)'] = "dx(f)*dy(g) - dy(f)*dx(g)"
problem.substitutions['u']      = "-dy(psi1)"
problem.substitutions['v']      = "dx(psi1)"

# equations of motion
problem.add_equation("dt(q1) + U*dx(q1) + (beta + Kd**2*U)*dx(psi1) - D*d(q1,x=2) - D*d(q1,y=2)= - J(psi1,q1)",condition="(nx!=0) or (ny!=0)")
problem.add_equation("psi1 = 0",condition="(nx==0) and (ny==0)")
problem.add_equation("dt(q2) - U*dx(q2) + (beta - Kd**2*U)*dx(psi2) - D*d(q2,x=2) - D*d(q2,y=2) = - J(psi2,q2)",condition="(nx!=0) or (ny!=0)")
problem.add_equation("psi2 = 0",condition="(nx==0) and (ny==0)")

## Create the solver

In [ ]:
solver =  problem.build_solver(de.timesteppers.RK443)

solver.stop_sim_time = T
solver.stop_wall_time = np.inf
solver.stop_iteration = np.inf

initial_dt = 0.2*Lx/Nx
max_dt = 1e-1
cfl = flow_tools.CFL(solver,initial_dt,safety=0.8)
cfl.add_velocities(('u','v'))

## Initial condition

The evolution of the instability does not depend strongly on the choice of initial condition. Here we have chosen an initial small perturbation to the PV in each layers.

In [ ]:
x,y = domain.grids(scales=domain.dealias)

q1_init = domain.new_field()
q1_init.set_scales(domain.dealias,keep_data=False)
q2_init = domain.new_field()
q2_init.set_scales(domain.dealias,keep_data=False)

eps = .3 # initial perturbation strength

q1_init['g'] = eps*np.cos(4*np.pi/Lx*x)*np.exp(-(x**2+y**2)/8)
q2_init['g'] = -eps*np.cos(4*np.pi/Lx*x)*np.exp(-(x**2+y**2)/8)

## Boundary value problem for streamfunction

We are specifying the initial potential vorticity, from which we need to derive the initial streamfunction. We do this by solving a Boundary Value Problem for the initial streamfunction.

In [ ]:
init_problem = de.LBVP(domain, variables=['psi1','psi2'])

# declare the problem parameters
init_problem.parameters['q1_init']   = q1_init
init_problem.parameters['q2_init']   = q2_init
init_problem.parameters['Kd']        = Kd

# boundary value problem
init_problem.add_equation("d(psi1,x=2) + d(psi1,y=2) + Kd**2*(psi2-psi1)/2 = q1_init",condition="(nx!=0) or (ny!=0)")
init_problem.add_equation("d(psi2,x=2) + d(psi2,y=2) - Kd**2*(psi2-psi1)/2 = q2_init",condition="(nx!=0) or (ny!=0)")
init_problem.add_equation("psi1 = 0",condition="(nx==0) and (ny==0)")
init_problem.add_equation("psi2 = 0",condition="(nx==0) and (ny==0)")

In [ ]:
# solve for the initial state
init_solver = init_problem.build_solver()
init_solver.solve()
psi1_init = init_solver.state['psi1']
psi1_init.set_scales(domain.dealias, keep_data=False)
psi2_init = init_solver.state['psi2']
psi2_init.set_scales(domain.dealias, keep_data=False)

## Plot initial conditions

We can now visualize the streamfunction and potential vorticity. The *total* streamfunction is given by the sum of the base state and the initial perturbation

$$
\psi_1 = \overline{\psi_1} + \psi_1', \qquad \text{where} \qquad \overline{\psi_1} = - U \, y,
$$

$$
\psi_2 = \overline{\psi_2} + \psi_2', \qquad \text{where} \qquad \overline{\psi_2} = U \, y.
$$

Likewise, the *total* potential vorticity is

$$
q_1 = \overline{q_1} + q_1', \qquad \text{where} \qquad \overline{q_1} = \left( \beta + K_d^2 U \right) \, y,
$$

$$
q_2 = \overline{q_2} + q_2', \qquad \text{where} \qquad \overline{q_2} = \left( \beta - K_d^2 U \right) \, y.
$$

In [ ]:
# Plot the initial conditions

xx,yy = np.meshgrid(x,y,indexing='ij')

fig = plt.figure(figsize=(16,16))

ax  = fig.add_subplot(2,2,1)
cax = ax.contourf(xx, yy, psi1_init['g'],8,cmap=plt.cm.coolwarm)
dax = ax.contour(xx, yy, psi1_init['g'],8,colors='k')
ax.set(title='$\Psi_1$ (t = 0)')
cbar = fig.colorbar(cax)

ax  = fig.add_subplot(2,2,2)
cax = ax.contourf(xx, yy, q1_init['g']+(beta+Kd**2*U)*yy,8,cmap=plt.cm.PuOr)
dax = ax.contour(xx, yy, q1_init['g']+(beta+Kd**2*U)*yy,8,colors='k')
ax.set(title='$q_1$ (t = 0)')
cbar = fig.colorbar(cax)

ax  = fig.add_subplot(2,2,3)
cax = ax.contourf(xx, yy, psi2_init['g'],8,cmap=plt.cm.coolwarm)
dax = ax.contour(xx, yy, psi2_init['g'],8,colors='k')
ax.set(title='$\Psi_2$ (t = 0)')
cbar = fig.colorbar(cax)

ax  = fig.add_subplot(2,2,4)
cax = ax.contourf(xx, yy, q2_init['g']+(beta-Kd**2*U)*yy,8,cmap=plt.cm.PuOr)
dax = ax.contour(xx, yy, q2_init['g']+(beta-Kd**2*U)*yy,8,colors='k')
ax.set(title='$q_2$ (t = 0)')
cbar = fig.colorbar(cax)

## Analysis

Since our model is evolving the streamfunction `psi` in each layer, we would like to save these fields, along with the potential vorticity `q` and velocities `u` and `v` every few timesteps. These will be saved in a directory `analysis_dir`.  

In addition to snapshots of the streamfunction, velocity, and potential vorticity, we will save time series of the kinetic energy and available potential energy of the perturbations:

$$
KE = \frac{1}{2} \int | \boldsymbol{\nabla} \psi_1' |^2 + | \boldsymbol{\nabla} \psi_2' |^2 \, dA
$$

$$
APE = \int K_d^2 \, \left( \psi_1' - \psi_2' \right)^2 \, dA
$$

In [ ]:
# first remove any existing analysis folders
analysis_dir = 'analysis'
import os
import shutil

if os.path.exists(analysis_dir):
    print("Overwriting existing directory " + analysis_dir)
    shutil.rmtree(analysis_dir)

# Create analysis tasks

analysis = solver.evaluator.add_file_handler(analysis_dir, sim_dt=max_dt, max_writes=100)
analysis.add_system(solver.state, layout='g')
analysis.add_task("d(psi1,x=2)+d(psi1,y=2)+Kd**2*(psi2-psi1)", layout='g', name='q1')
analysis.add_task("d(psi2,x=2)+d(psi2,y=2)-Kd**2*(psi2-psi1)", layout='g', name='q2')
analysis.add_task("-dy(psi1)", layout='g', name='u1')
analysis.add_task(" dx(psi1)", layout='g', name='v1')
analysis.add_task("-dy(psi2)", layout='g', name='u2')
analysis.add_task(" dx(psi2)", layout='g', name='v2')
analysis.add_task("integ(0.5*(dx(psi1)*dx(psi1)+dy(psi1)*dy(psi1)+dx(psi2)*dx(psi2)+dy(psi2)*dy(psi2)))",name='KE')
analysis.add_task("integ(Kd**2*(psi1-psi2)*(psi1-psi2))", name='APE')

## Run the simulation

We are now ready to run the simulation. We call a variables `psi1` and `psi2` and give them the initial values `psi1_init` and `psi2_init`, respectively. Then we run through the main time loop, saving data as we go using the analysis tasks that we defined.  

In [ ]:
import time
psi1 = solver.state['psi1']
psi1.set_scales(domain.dealias,keep_data=False)
psi1['g'] = psi1_init['g']
psi2 = solver.state['psi2']
psi2.set_scales(domain.dealias,keep_data=False)
psi2['g'] = psi2_init['g']

# Main loop
start_time = time.time()
while solver.ok:
    dt = min(max_dt,cfl.compute_dt())
    solver.step(dt)
    if solver.iteration % 100 == 0:
        print('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))
end_time = time.time()
print('Runtime:', end_time-start_time)

## Merge the analysis files and read in the data

Dedalus saves the analysis data (snapshots of the flow etc) to a number of subdirectories in the directory `analysis`. In order to plot figures and make movies, we need to merge these files together into a single data set and then read it into memory.

In [ ]:
# Merge the analysis files

import subprocess
from dedalus.tools import post
import pathlib
post.merge_process_files(analysis_dir, cleanup=True)
set_paths = list(pathlib.Path(analysis_dir).glob("analysis_s*.h5"))
post.merge_sets(analysis_dir + "/analysis.h5", set_paths, cleanup=True)

In [ ]:
# Read in the data

import h5py

file = h5py.File(analysis_dir + '/analysis.h5','r')
t    = file['/scales/sim_time'][:]
X    = file['/scales/x/1.0'][:]
Y    = file['/scales/y/1.0'][:]
psi1 = file['tasks']['psi1'][:]
q1   = file['tasks']['q1'][:]
psi2 = file['tasks']['psi2'][:]
q2   = file['tasks']['q2'][:]
APE  = file['tasks']['APE'][:]
KE   = file['tasks']['KE'][:]

file.close()

APE = APE[:,0,0]
KE  = KE[:,0,0]

## Plot snapshots of the PV

Now we can plot snapshots of the (total) potential vorticity and save them to a new directory called `plots`. Once we have created the snapshots, we can stitch them together into a movie.

You can easily adapt this code if you would like to visualize the streamfunction or the velocity fields instead.

In [ ]:
# first remove any existing plot folders
plot_dir = "plots"
import os
import shutil

if os.path.exists(plot_dir):
    print("Overwriting existing directory " + plot_dir)
    shutil.rmtree(plot_dir)

os.makedirs(plot_dir)

XX,YY = np.meshgrid(X,Y,indexing='ij')

for i in range(len(t)):

    fig = plt.figure(figsize=(16, 8))
    ax  = fig.add_subplot(1,2,1)

    cax = ax.contourf(XX, YY, q1[i,:,:]+(beta+Kd**2*U)*YY,8,cmap=plt.cm.PuOr)
    dax = ax.contour(XX, YY, q1[i,:,:]+(beta+Kd**2*U)*YY,8,colors='k')
    ax.set(title='q1: t = ' + str(round(t[i],4)))
    cbar = fig.colorbar(cax)

    ax  = fig.add_subplot(1,2,2)
    cax = ax.contourf(XX, YY, q2[i,:,:]+(beta-Kd**2*U)*YY,8,cmap=plt.cm.PuOr)
    dax = ax.contour(XX, YY, q2[i,:,:]+(beta-Kd**2*U)*YY,8,colors='k')
    ax.set(title='q2: t = ' + str(round(t[i],4)))
    cbar = fig.colorbar(cax)

    fig.savefig(plot_dir + '/snap_{:0>3d}.png'.format(i),bbox_inches='tight')
    plt.close()
    if i%10 == 0:
        print('Saved figure',i)

print('Saved snapshots')

## Viewing the simulation

Below are two different ways you can view the output of your simulation. You can choose which option to use based on what works on your particular machine and web browser.

### Option 1: Combine the images into a GIF

In [ ]:
# Option 1: combine the images into a GIF

from PIL import Image
import glob

def save_gif(image_filenames,gif_filename):
  frames = []
  imgs = sorted(glob.glob(image_filenames))
  for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

  frames[0].save(gif_filename, format='GIF',
    append_images=frames[1:],
    save_all=True,
    duration=100, loop=0)

gif_filename = "exp_U_" + str(U) + "_Kd_" + str(Kd) +".gif"
save_gif(plot_dir + "/*.png",gif_filename)
print('Created ' + gif_filename)

To play the GIF, simply double click the file in the file browser on the left. This will open up an image browser in the Colab window.

### Option 2: Combine the images into an MP4

In [ ]:
# Option 2: combine the images into a MP4

def save_mp4(image_filenames,mp4_filename,framerate):
    os.system('ffmpeg -r ' + str(framerate) + ' -i ' + image_filenames + ' -vb 20M -vcodec mpeg4 -y ' + mp4_filename)

# make movie
mp4_filename = "exp_U_" + str(U) + "_Kd_" + str(Kd) +".mp4"
save_mp4(plot_dir + "/snap_%03d.png",mp4_filename,10)
print('Created ' + mp4_filename)

To play the movie, you can download it from the file browser on the left. The recommended movie player is VLC, which you can download [here](https://www.videolan.org/vlc/).

Alternatively, you can run the code cell below to play the movie in the web browser. NOTE: this should work if you are running Safari on a Mac, but it may not work on your particular machine or internet browser.

In [ ]:
# play movie in the web browser
from IPython.display import HTML
from base64 import b64encode
mp4 = open(mp4_filename,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
    <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Available potential energy and kinetic energy



In [ ]:
fig = plt.figure(figsize=(10, 5))
ax  = plt.axes()
ax.plot(t,APE,label="APE")
ax.plot(t,KE,label="KE")

plt.xlim([0,T])
plt.title(r'Available Potential Energy / Kinetic Energy',fontsize=16)
plt.xlabel(r'Model time',fontsize=16)

plt.legend(loc='lower left',fontsize=16).draw_frame(True)

# Exercises

In each of the following experiments, run the simulation and ask yourself the following questions:

- Is the shear flow velocity larger than the threshold shear $U_s = \beta / K_d^2$?
- What is the base state potential vorticity gradient in each layer? Does it change sign?  
- What direction do Rossby waves propagate in each layer?
- Describe and explain what happens to the APE and KE as functions of time.
- Describe qualitatively how the phases in each layer are related to one another.

### Experiment 1: no shear flow, no coupling between layers

In the first experiment, there is no shear flow (`U = 0`) and no coupling between the layers via vortex stretching or squeezing (`Kd = 0`).

### Experiment 2: shear flow, no coupling between layers

Now let's turn on the shear flow. Change the value of the shear flow velocity to `U = 1`. (This gives a base state flow of $U = 1$ in the upper layer and $U = -1$ in the lower layer.) However, we will still ignore coupling between the layers (`Kd = 0`).

### Experiment 3: shear flow, coupling between layers

Finally, let's turn on the shear flow and the coupling between the layers. Change the value of the shear flow velocity to `U = 1` and the inverse deformation scale to `Kd = 2`.